# Packages

In [50]:
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, B, C, D, E, F, G, H, J
import itertools
import numpy as np

In [51]:
%%capture
!pip install python-sat

In [52]:
from pysat.examples.fm import FM
from pysat.formula import WCNF
from itertools import combinations
import math

# Prepare indices

In [53]:
class Model:
    def __init__(self, n_neurons):
        self.n_neurons = n_neurons
        
    
class DataProperties:
    def __init__(self, n_features, train_size):
        self.n_features = n_features
        self.n_train = train_size

        self.X = None
        self.Y = None
        
        self.prop_features = None
        self.prop_weights = np.array([i + 1 for i in range(n_features)])

        self.features_combinations = None
        self.cartesian_products = None

        self.positive_clauses = []
    
    def encode_propositional_features(self):
        self.propositional_features = np.array(
            [[i*self.n_features + j + max(self.prop_weights) + 1 for j in range(self.n_features)] for i in range(self.n_train)]
        )

    def encode_features_combinations(self):

        majority_size = int(np.ceil(self.n_features / 2))
        assert(majority_size == np.floor(self.n_features / 2) + 1)
        self.features_combinations = list(combinations(self.prop_weights, majority_size))

    def remove_redundant(self, elements):
        res = []
        for e in elements:
            if not (e in res):
                res.append(e)
        return res

    def encode_cartesian_products(self):
        prods = list(itertools.product(*self.features_combinations))
        prods = [np.unique(combo).tolist() for combo in prods]
        self.cartesian_products = self.remove_redundant(prods)

    def prepare_predfinal_clauses(self):
        predfinal_clauses = []
        for product in self.cartesian_products:
            bracket = []
            for val in product:
                bracket.append([val, -val])
            predfinal_clauses.append(bracket)
        return predfinal_clauses


    def encode_positive_clauses(self):
        # Each element of optimized_products represents:
        # [1, 2] --> (w1 == x_i1) V (w2 == x_i2). But we have not transfered to CNF yet
        # To do that: [(~w1 V x_i1) & (w1 V ~x_i1)] V [(~w2 V x_i2) & (w2 V ~x_i2)]
        # which is 'encoded' as [[1, -1], [2, -2]]

        predfinal_clauses = self.prepare_predfinal_clauses()

        # And now we are creating final clauses
        for predfinal_clause in predfinal_clauses:
            clauses = list(itertools.product(*predfinal_clause))
            
            for clause in clauses:
                self.positive_clauses.append(clause)

        








model = Model(n_neurons = 5)

data = DataProperties(
    n_features = model.n_neurons,
    train_size = 30
)

## Encode propositional features

In [54]:
# define propositional variables for X
# propositional features cannot have same indices as w --> + 1 + max(w)
data.encode_propositional_features()

In [55]:
# Now prepare all cartesian products (for the case whey y == 1)
data.encode_features_combinations()
data.encode_cartesian_products()

## Encode clauses indices for y == 1 case

In [56]:
data.encode_positive_clauses()

## Encode clauses indices for y == -1

In [ ]:
data.encode_negative_clauses()

# Encode clauses for y == -1

In [ ]:
features_combinations

In [ ]:
ws_xs_list = []
for combo in features_combinations:
    # print(combo)
    symbols = []
    for i in combo:
        symbols.extend([[f'w {i}', f'~ w {i}']])
        symbols.extend([[f'x {i}', f'~ x {i}']])
    ws_xs_list.append(symbols)

ws_xs_list

In [ ]:
print(*itertools.product(*ws_xs_list[0]))

In [ ]:
negative_clauses = []

for combination in ws_xs_list:
    for clause in itertools.product(*combination):
        # print(clause)
        negative_clauses.append(clause)

In [ ]:
# Here, w1 means 1st weight, x1 - first feature of the object
len(set(negative_clauses)), len(negative_clauses)

# Preparing dataset

In [ ]:
def check_if_2nd_feature_true(x):
    return x[1] == True

def check_features(x):
    return (x[1] == True)

def prepare_dataset(n_train_, n_features_, func_):
    sample_arr = [True, False]

    X = np.random.choice(sample_arr, size = (n_train_, n_features_))
    Y = [func_(x) for x in X]
    return X, Y

# Randomly prepare dataset
data_function = check_features  # check_if_2nd_feature_true

X, Y = prepare_dataset(
    n_train_ = n_train,
    n_features_ = n_features,
    func_ = data_function
)

In [ ]:
print(X)
print(Y)

In [ ]:
for x, y in zip(X, Y):
    print(x, data_function(x), y)
    assert(data_function(x) == y)

# Encoding into clauses

In [ ]:
positive_clauses

In [ ]:
propositional_features[0]

In [ ]:
len(positive_clauses), positive_clauses

# Encoding soft clauses

In [ ]:
def calc_update(letters, x, w):
    # ['~', 'x', '2']
    x_or_w = ''
    to_put_minus = 1

    if letters[0] == '~':
        x_or_w = letters[1]
        to_put_minus = -1
    else:
        x_or_w = letters[0]
        to_put_minus = 1
    
    index = int(letters[-1])
    if x_or_w == 'x':
        return int(to_put_minus * x[index - 1])
        
    if x_or_w == 'w':
        return int(to_put_minus * w[index - 1])

# w[i] is a (i+1)th weight (i.e. w = (1, 2, 3), 1 = w[0], 2 = w[1], 3 = w[2])
wcnf_form = WCNF()

for i in range(len(Y)):
    
    y_i = Y[i]

    if y_i == True:
        print(f'Encoding for y True')
        for positive_clause in positive_clauses:
            # if (1, -2) clause --> [(w[0] V ~x[i][0]) V (~w[1] V x[i][1])] clause
            clause_to_add = []
            for index in positive_clause:
                abs_index = abs(index)
                prop_weight = w[abs_index - 1]  # variable, where weight is encoded
                prop_feature = propositional_features[i][abs_index - 1]  # feature, that is encoded

                if index < 0:
                    clause_to_add.extend([int(-prop_weight), int(prop_feature)])
                else:
                    clause_to_add.extend([int(prop_weight), int(-prop_feature)])
            # print(f'Clause: {clause_to_add}')
            wcnf_form.append(clause_to_add, weight = 1)

    elif y_i == False:
        print(f'Encoding for negative y')
        for negative_clause in negative_clauses:
            # ('w 1', 'x 1', '~ w 2', '~ x 2')

            clause_to_add = []
            for symbol in negative_clause:
                # symbol is a string, like "w 1" or "~ x 2"
                letters = symbol.split(' ')
                clause_to_add.append(int(calc_update(letters, propositional_features[i], w)))
            # print(f'Clause: {clause_to_add}')
            wcnf_form.append(clause_to_add, weight = 1)

# Encoding hard clauses

In [ ]:
print(X)
print(propositional_features)

In [ ]:
for i in range(len(X)):
    for j in range(len(X[0])):
        if X[i][j] == True:
            wcnf_form.append([int(propositional_features[i][j])])
        else:
            wcnf_form.append([int(-propositional_features[i][j])])

# Compute solution

In [ ]:
wcnf_form.hard

In [ ]:
fm = FM(wcnf_form,verbose=10)
fm.compute()

In [ ]:
fm.model

In [ ]:
for l in fm.model:
    print(l)

# Checking the solution on the training data

In [ ]:
len(w)
weights_solution = fm.model[:len(w)]
weights_solution = [val > 0 for val in weights_solution]
print(weights_solution)

# Now, we encode weights into {0, 1} and X features into {0, 1}

In [ ]:
a = np.array(X)
a[a == True]

In [ ]:
X_decoded = np.array(X, dtype = 'int')
Y_decoded = np.array(Y, dtype = 'int')
w_decoded = np.array(weights_solution, dtype = 'int')

In [ ]:
# X [4 x 3] * W [3 x 1] = Y^ [4 x 1]
np.sign(np.dot(X_decoded, w_decoded)), Y_decoded

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_decoded, np.sign(np.dot(X_decoded, w_decoded)))

In [ ]:
X_decoded

# Task

In [ ]:
from pysat.formula import CNF
from pysat.formula import WCNF

In [ ]:
n_neurons = 3
n_train = 3
n_features = n_neurons

assert(n_neurons % 2 == 1)  # odd number of neurons is allowed



In [ ]:
print(X)
print(Y)

In [ ]:
# wcnf = WCNF()

# wcnf.append([-1, 2]) 
# wcnf.append([-2, 3])
# wcnf.append([-3, True])
# wcnf.append([4, True])

# print(wcnf.soft)
# print(wcnf.hard)
# print(wcnf.wght)

# fm = FM(wcnf,verbose=10)
# fm.compute()

In [ ]:
# def encode_features(X):
#     n = len(X)
#     m = len(X[0])
#     print(f'Shape of X: n = {n}, m = {m}')
#     prop_features = [[i * n + j + 1 for j in range(m)] for i in range(n)]
#     return np.array(prop_features)


# Encoding weights
prop_weights_indices = [i for i in range(n_neurons)]

In [ ]:
prop_weights_indices

In [ ]:
group_size = int(math.ceil(n_neurons / 2))
assert group_size == math.floor(n_neurons / 2 + 1)

In [ ]:
combos = list(combinations(prop_weights_indices, group_size))
print(combos[:3])

In [ ]:
cartesian_prod = list(itertools.product(*combos))
print(cartesian_prod)

In [ ]:
clauses_cartesian_products = []
for c in cartesian_prod:
    print(f'Combo: {c}')
    l = [[f'{symb}', f'~{symb}'] for symb in c]
    l_cart_prod = list(itertools.product(*l))
    clauses_cartesian_products += l_cart_prod
    print(l)
    print(l_cart_prod)

In [ ]:
clauses_cartesian_products

In [ ]:
import numpy as np

print(len(clauses_cartesian_products))
len(np.unique(clauses_cartesian_products))
len(set(clauses_cartesian_products))

In [ ]:
def encode_prop_features(X):
    n = len(X)
    m = len(X[0])
    prop_features = []
    for i in range(n):
        vals = [i * n + j + len(prop_weights_indices) for j in range(m)]
        prop_features.append(vals)
    

    for i in range(n):
        for j in range(m):
            print(f'X[{i}][{j}] is encoded as {prop_features[i][j]}')

    return prop_features

prop_features = encode_prop_features(X)
print(prop_features)

In [ ]:
unique = [1, 2]
print(get_combinations(unique))

In [ ]:
clauses_cartesian_products

In [ ]:
def get_combinations(unique_combo):
    ws_and_xs = []
    for numb in unique_combo:
        ws_and_xs.append(f'w {numb}')
        ws_and_xs.append(f'x {numb}')
    
    ws_and_xs = [[f'{symb}', f'~{symb}'] for symb in ws_and_xs]
    return list(itertools.product(*ws_and_xs))

def construct_clause_positive(unique_clause_symbols, ):
    clause = []
    # '1' signifies ~w1 V x_i1
    # '~1' signifies w1 V ~x_i1
    for symbol in unique_clause_symbols:  # ('0', '~2', '1')
        if not '~' in symbol:
            # then (e.f. for '1') its not(w_1) V (x_i1)
            index = int(symbol)
            clause.append(-prop_weights_indices[index])
            clause.append(prop_features[i][index])
        elif '~' in symbol:
            # then (e.f. for '1') its (w_1) V not(x_i1)
            symbol = symbol.replace('~', '')
            index = int(symbol)
            clause.append(prop_weights_indices[index])
            clause.append(-prop_features[i][index])
    optimized_clause = list(set(clause))
    return optimized_clause
    
def construct_clause_negative(unique_combo):
    all_combinations = get_combinations(unique_combo)

    clauses = []
    for combination in all_combinations:
        clause = []
        for symbol in combination:
            x_or_w, number = symbol.split(' ')[0], int(symbol.split(' ')[1])
            if x_or_w == 'w':
                clause.append(prop_weights_indices[number])
            if x_or_w == '~w':
                clause.append(-prop_weights_indices[number])
            if x_or_w == 'x':
                clause.append(prop_features[i][number])
            if x_or_w == '~x':
                clause.append(-prop_features[i][number])
        clauses.append(clause)
    return clauses


wcnf = WCNF()
for i in range(len(X)):
    if Y[i] is True:
        for c in clauses_cartesian_products:
            # ('0', '0', '~1')
            # Here we optimize - we dont need to add identical clauses
            unique_clause_symbols = set(c)  # ('0', '0', '1') --> ('0', '1')
            
            # print(set(c), c)
            clause = construct_clause_positive(unique_clause_symbols)
            wcnf.append(clause, weight = 1)
            # print(f'Our clause: {clause}')
    
    elif Y[i] is False:
        # print(f'Y is false')
        for c in cartesian_prod: # (1, 2) (2, 3) (1, 3) --> (1, 2, 1), (1, 2, 3), (1, 3, 1), ...
            # print(f'combo: {c}')
            unique_combo = set(c)  # (1, 2) --> [(~w1 & x_i1) V (w1 & ~x_i1)] V [(~w2 & x_i2) V (w2 & ~x_i2)]
            # print(f'unique combo: {set(unique_combo)}')

            clauses = construct_clause_negative(unique_combo)
            for c in clauses:
                wcnf.append(c, weight = 1)
    # print(clause)
    

In [ ]:
get_combinations([0, 1])

In [ ]:
construct_clause_negative([0, 1])

In [ ]:
cartesian_prod

In [ ]:
len(wcnf.soft)

In [ ]:
# Weighted CNF form
wcnf = WCNF()


for x, y in zip(X, Y):
    print(f'Encoding x = {x}, y = {y}')
    
    # At that step, we have combinations of weight indices
    # For example, for 3 weights
    # [(w0 == x_i0) & (w1 == x_i1)] V [(w0 == x_i0) & (w2 == x_i2)] V [(w1 == x_i1) & (w2 == x_i2)]

    # Each combination in cartesian_prod(w_combinations) - is a special clause:
    # For example, for n_weights == 3, after cartesian product, all combinations:
    # (001) (002) (021) (022) (101) (102) (121) (122)
    # Which means (for the first clause (001)):
    # (w_0 == x_i0) | (w_0 == x_i0) | (w_1 == x_i1)

    # However, we still did not expand the equivalence (==):
    # [(~w_0 V x_i0) & (w_0 V ~x_i0)] V [(~w_0 V x_i0) & (w_0 V ~x_i0)] V [(~w_1 V x_i1) & (w_1 V ~x_i1)]

    # And this also has to be transformed into CNF:
    # Do substitution:
    # [(0) & (~0)] V [(0) & (~0)] V [(1) & (~1)]
    # Where 0: ~w_0 V x_i0, ~0: w_0 V ~x_i0
    # We have to transfer (001) -> [[0, ~0], [0, ~0], [1, ~1]] and make a cartesian product of it

    

    

    # for w_combination in combinations(propositional_weights, group_size):
    #     # print(c, c[0], c[1])
    #     # print(propositional_weights.index(c[0]), propositional_weights.index(c[1]))
    #     print(f'weight combination = {w_combination}')
        
    #     for weight in w_combination:
    #         # index of weight --> the same as index of feature of current object x_i in the X dataset
    #         w_index = propositional_weights.index(weight)
    #         x_feature = x[w_index]
    #         print(f'w_{w_index}, x_j = {x_feature}')

    #         # Our clause is - weight == x_feature (equivalence)
    #         # which is (not(weight) V x_feature) and (not(x_feature) V weight)
            
    #         first_clause = [-weight, x_feature]
    #         second_clause = [weight, -x_feature]
    #         wcnf.append



In [ ]:
# No hard clauses are introduced.
# Soft clauses - have weight 1, if the prediction of the network is correct
# for a given instance (x_i, y_i) in a dataset

In [ ]:
# Encode this into clauses
for x_i, y_i in zip(X, Y):
    print(f'Encoding x_i = {x_i}, y_i = {y_i}')
    